In [ ]:
import coco_dataset
import pathlib
import show_utils
import matplotlib.pyplot as plt
from torchvision import transforms
import torchvision.transforms.functional as F
import torch
import common


In [ ]:
inp_size = (368, 368)
targ_size = (46, 46)
coco_dataset = coco_dataset.CocoKeypoints(
    root=str(pathlib.Path("../coco") / "images" / "train2017"),
    annFile=str(pathlib.Path("../coco") / "annotations" / "annotations" / "person_keypoints_train2017.json"),
    input_transform=transforms.Compose([
        transforms.Resize(inp_size),
        transforms.ToTensor(),
        transforms.ConvertImageDtype(torch.float32),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]),
    targ_size=targ_size)
i = 0


In [ ]:
i = 5

In [ ]:
sample = coco_dataset[i]
input_image, pafs, heatmaps, mask_out, keypoints, image, target = sample

show_utils.show_coco(image, target, coco_dataset.coco, draw_bbox=False)

print(f"sample: {i}")
i += 1

In [ ]:
import post
image_size = image.size
kpt_groups = post.post_process(heatmaps, pafs)

if kpt_groups:
    keypoints_post = post.coco_format(kpt_groups, image_size)
    show_utils.draw_keypoints(image,keypoints_post, connectivity=common.connect_skeleton, keypoint_color="orange", line_color="magenta")
else:
    print("NO DETECTIONS")

In [ ]:
# %matplotlib widget

# show_utils.show_heatmaps_combined(heatmaps)

# show_utils.show_heatmaps(heatmaps)

# show_utils.show_pafs(pafs)

# show_utils.show_pafs_quiver(pafs, keypoints, size=targ_size)

# plt.figure()
# show_utils.show_pafs_quiver_combined(pafs, size=targ_size)

# show_utils.show_tensors(mask_out)